## CGM2.1 Modellling 

run both CGM2.1 Calibration and Fitting Operations 


In [10]:
import logging 
from argparse import Namespace

# pyCGM2 libraries
import pyCGM2
from pyCGM2 import enums
from pyCGM2.Model.CGM2.coreApps import cgm2_1,cgmUtils
from pyCGM2.Tools import btkTools
from pyCGM2.Utils import files
from pyCGM2.Eclipse import vskTools

In [18]:
# DATA

DATA_PATH = "C:\\Users\\HLS501\\Documents\\VICON DATA\\pyCGM2-Data\\Datasets Tests\\didier\\08_02_18_Vincent Pere\\"
DATA_PATH_OUT = DATA_PATH
calibrateFilenameLabelled  = "08_02_18_Vincent_Pere_Statique_000_MOKKA.c3d"
reconstructFilenameLabelled = "08_02_18_Vincent_Pere_Gait_000_MOKKA.c3d"




In [19]:
# 
args = Namespace()
args.leftFlatFoot = None
args.rightFlatFoot = None
args.markerDiameter = None
args.pointSuffix = None
args.fileSuffix = None
args.mfpa = None
args.proj = None
args.check = True

# --------------------------GLOBAL SETTINGS ------------------------------------
# global setting ( in user/AppData)
settings = files.openJson(pyCGM2.CONFIG.PYCGM2_APPDATA_PATH,"CGM2_1-pyCGM2.settings")


# settings overloading
argsManager = cgmUtils.argsManager_cgm(settings,args)
leftFlatFoot = argsManager.getLeftFlatFoot()
rightFlatFoot = argsManager.getRightFlatFoot()
markerDiameter = argsManager.getMarkerDiameter()
pointSuffix = argsManager.getPointSuffix("cgm2_1")
momentProjection =  argsManager.getMomentProjection()
mfpa = argsManager.getManualForcePlateAssign() 


# Translator
translators = files.getTranslators(DATA_PATH,"CGM2_1.translators")
if not translators: translators = settings["Translators"]
    
 

## Calibration

In [21]:
hjcMethod = settings["Calibration"]["HJC"]

required_mp = {
    "Bodymass": 70,
    "Height": 1700,
    "LeftLegLength": 890,
    "RightLegLength": 890,
    "LeftKneeWidth": 150,
    "RightKneeWidth": 150,
    "LeftAnkleWidth": 88,
    "RightAnkleWidth": 99,
    "LeftSoleDelta": 0,
    "RightSoleDelta": 0
}
optional_mp = {
    "InterAsisDistance": 0,
    "LeftAsisTrocanterDistance": 0,
    "LeftTibialTorsion": 0,
    "LeftThighRotation": 0,
    "LeftShankRotation": 0,
    "RightAsisTrocanterDistance": 0,
    "RightTibialTorsion": 0,
    "RightThighRotation": 0,
    "RightShankRotation": 0,
}


In [22]:
model,acqStatic = cgm2_1.calibrate(DATA_PATH,calibrateFilenameLabelled,translators,required_mp,optional_mp,
              leftFlatFoot,rightFlatFoot,markerDiameter,hjcMethod,
              pointSuffix) 

## Fitting

In [24]:
#Fitting

acqGait = cgm2_1.fitting(model,DATA_PATH, reconstructFilenameLabelled,
                    translators,
                    markerDiameter,
                    pointSuffix,
                    mfpa,momentProjection)
    
# writer
if fileSuffix is not None:
    c3dFilename = str(reconstructFilenameLabelled[:-4]+"-modelled-"+fileSuffix+".c3d")
else:
    c3dFilename = str(reconstructFilenameLabelled[:-4]+"-modelled.c3d")

btkTools.smartWriter(acqGait, str(DATA_PATH_OUT+c3dFilename))
logging.info("c3d file (%s) generated" %(c3dFilename) )
    
            